In [81]:
library(httr)
library(data.table)
library("jsonlite")
library("tibble")
library("tidyr")
library("dplyr")

# curl query:
# curl "https://api.openweathermap.org/data/2.5/onecall?lat=51.509865&lon=-0.118092&exclude=minutely,hourly,alerts&appid=65d4508050d5008b768b660a688651ad" | python -mjson.tool
# Turkey Fields: 40.137442, 28.383499
    
api_key <- "appid=65d4508050d5008b768b660a688651ad"
base_url <- "https://api.openweathermap.org/data/2.5/onecall?"
url_settings <- "&exclude=minutely,hourly,alerts"

#Turkey Fields:
lat <- "&lat=40.137442"
lon <- "&lon=28.383499"
#Kassow, Germany:
#lat <- "&lat=53.881579"
#lon <- "&lon=12.072763"

# complete_url variable to store complete url address
complete_url = paste(base_url,api_key,lat,lon,url_settings, sep = "", collapse = "")
print(complete_url)
    
response <- httr::GET(complete_url)

# print header of the response:
#headers(response)

# JSON automatically parsed into names list:
cont_raw <-  httr::content(response)
cont_text <- httr::content(response, "text")
cont_list <- httr::content(response, "parsed")

#typeof(response)
typeof(cont_raw)
typeof(cont_text)
typeof(cont_list)

data_raw_ugly <- jsonlite::fromJSON(cont_text)

# solution from https://www.r-bloggers.com/2018/10/converting-nested-json-to-a-tidy-data-frame-with-r/
# flatten the nested list:
data_raw <- tibble::enframe(unlist(cont_raw))
#head(data_raw,10)

# separate() turns a single character column into multiple columns
# The data from the "name" col is split at "." into 3 columns (x1,x2,x3) 
data_sep <-
  data_raw %>% tidyr::separate(name, sep = "\\.",into = c(paste0("name", 1:3)), fill = "right")
head(data_sep,60)



[1] "https://api.openweathermap.org/data/2.5/onecall?appid=65d4508050d5008b768b660a688651ad&lat=40.137442&lon=28.383499&exclude=minutely,hourly,alerts"


[1] "list"

[1] "character"

[1] "list"

name1,name2,name3,value
<chr>,<chr>,<chr>,<chr>
lat,NA,NA,40.1374
lon,NA,NA,28.3835
timezone,NA,NA,Europe/Istanbul
timezone_offset,NA,NA,10800
current,dt,NA,1649860524
current,sunrise,NA,1649820701
current,sunset,NA,1649868118
current,temp,NA,288.34
current,feels_like,NA,286.99


In [87]:
# Post-processing the data

data_filt <-
 data_sep %>%
  filter(
    (name1 == "lat") |
    (name1 == "lon") |
    (name1 == "timezone") |
    (name1 == "daily" & name2 == "dt") |
    (name1 == "daily" & name2 == "temp" & name3 == "day") |
    (name1 == "daily" & name2 == "humidity") |
    (name1 == "daily" & name2 == "dew_point") |
    (name1 == "daily" & name2 == "wind_speed") |
    (name1 == "daily" & name2 == "uvi") |
    (name1 == "daily" & name2 == "rain")
  )

data_filt


name1,name2,name3,value
<chr>,<chr>,<chr>,<chr>
lat,NA,NA,40.1374
lon,NA,NA,28.3835
timezone,NA,NA,Europe/Istanbul
daily,dt,NA,1649844000
daily,temp,day,287.46
daily,humidity,NA,38
daily,dew_point,NA,272.7
daily,wind_speed,NA,3.64
daily,uvi,NA,6.12
